In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
from datetime import datetime
pd.options.mode.chained_assignment=None
import tsfresh


。。

In [2]:

def data_preprocess(data,low=0.1):
    '''
    Preprocess data in order to calculate threshold
    Input:
        data: DataFrame
        columns: [TimeStr, Ia]
    Output:
        data: DataFrame
        columns: [time, Ia]
    '''

    data = data[['TimeStr', 'Ia']]
    data.loc[:,'TimeStr'] = pd.to_datetime(data['TimeStr'])
    data.reset_index(drop=True, inplace=True)#删除日期缺失值
    # data.loc[data['Ia'] > 1000, 'Ia'] = -1
    data.dropna(subset=['TimeStr'], inplace=True)
    if len(data) == 0:
        return data
    #处理日期重复
    data.loc[:,'TimeStr']=pd.to_datetime(data['TimeStr'].apply(lambda x: datetime.strftime(x, format='%Y-%m-%d %H:%M:%S')))
    data.drop_duplicates('TimeStr', inplace=True)
    data.reset_index(drop=True, inplace=True)
    #日期补全
    time_series = pd.date_range(data["TimeStr"].min(), data["TimeStr"].max(), freq='s')
    ts = pd.DataFrame(time_series, columns=["time"])
    data = data.merge(ts, how='right', left_on="TimeStr", right_on="time")
    data.drop(['TimeStr'], axis=1, inplace=True)
    data.sort_values(by='time', inplace=True)
    data.reset_index(drop=True, inplace=True)
    data['Ia'] = data['Ia'].rolling(window=5, min_periods=1, center=False).median()
    data.fillna(-1, inplace=True)
    data = data[['time', 'Ia']]
    '''
    data['time','Ia','no']
    '''
    data = data.drop(data[data['Ia'] <=low].index).reset_index(drop=True)  #删除-1 0
    if len(data)==0:
        print('电流值全部小于%s' % low)
    data['no']=pd.Series(range(len(data)))
    return data  



In [3]:
data=pd.read_csv(r'/home/wrj/data/10.9.130.99.csv')
data=data_preprocess(data)
data.tail()
# data=data[0:5000]

,time,Ia,no
78013,2018-07-24 23:59:54,5.52,78013
78014,2018-07-24 23:59:55,5.52,78014
78015,2018-07-24 23:59:56,5.52,78015
78016,2018-07-24 23:59:57,5.52,78016
78017,2018-07-24 23:59:58,5.52,78017


In [4]:
data.head()

,time,Ia,no
0,2018-07-24 00:00:00,5.560,0
1,2018-07-24 00:00:01,5.565,1
2,2018-07-24 00:00:02,5.570,2
3,2018-07-24 00:00:03,5.565,3
4,2018-07-24 00:00:04,5.565,4


波类

In [ ]:
class wave(object):
    
    

特征类

In [8]:
class funset(object):
   
    def __init__(self,data):
        self.len_standard_wave=300
        self.len_window=600
        self.data=data.copy()
        self.y={}
        self.data_y={}
        self.matrix_length=len(self.data)-self.len_window+1 #特征向量的行数 或 窗口的总个数
        self.result=pd.DataFrame({'window_no':range(self.matrix_length)})
       
    def data_process(self):
        for i in range(self.matrix_length):
            self.data_y.setdefault(i)
            self.y.setdefault(i)
            idx_end=i+self.len_window
            self.data_y[i]=data.iloc[i:idx_end,:]
            self.y[i]=data.loc[i:idx_end,'Ia']
        return self
        
    def longest_strike_above_mean(self):
        f=tsfresh.feature_extraction.feature_calculators.longest_strike_above_mean
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.result['longest_strike_above_mean']=res_l
        
        return res_l
        
    def longest_strike_below_mean(self):
        f=tsfresh.feature_extraction.feature_calculators.longest_strike_below_mean
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.result['longest_strike_below_mean']=res_l
        
        return res_l
    
    def sum_values(self):
        f=tsfresh.feature_extraction.feature_calculators.sum_values
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.result['sum_values']=res_l
        
        return res_l
    
    def ratio_beyond_r_sigma(self,r=0.3):
        f=tsfresh.feature_extraction.feature_calculators.ratio_beyond_r_sigma
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i],r))
        self.result['ratio_beyond_r_sigma']=res_l
            
        return res_l
    
    def absolute_sum_of_changes(self,r=0.3):
        f=tsfresh.feature_extraction.feature_calculators.absolute_sum_of_changes
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.result['absolute_sum_of_changes']=res_l
            
        return res_l
    
    def mean_second_derivative_central(self):
        f=tsfresh.feature_extraction.feature_calculators.mean_second_derivative_central
        res_l=[]
        for i in range(self.matrix_length):
            res_l.append(f(self.y[i]))
        self.result['mean_second_derivative_central']=res_l
            
        return res_l
    
    def standardscaler(self):
        f=StandardScaler()
        out=f.fit_transform(self.result.iloc[:,1:])
        
        return out


In [16]:
a=funset(data)
a.data_process()

In [17]:
a.sum_values()
a.absolute_sum_of_changes()
a.mean_second_derivative_central()
a.result.shape

(77419, 4)

In [18]:
a.result

,window_no,sum_values,absolute_sum_of_changes,mean_second_derivative_central
0,0,3262.355,2.230,-0.000004
1,1,3262.135,2.225,-0.000004
2,2,3261.910,2.220,0.000004
3,3,3261.680,2.215,0.000000
4,4,3261.455,2.215,0.000000
5,5,3261.230,2.215,0.000004
6,6,3261.005,2.210,0.000004
7,7,3260.780,2.210,0.000000
8,8,3260.560,2.205,0.000000
9,9,3260.350,2.210,-0.000004


In [13]:
a.standardscaler()

array([[ 8.66568020e-01, -5.76821561e-01, -6.48553392e-02],
       [ 8.65028802e-01, -5.78174342e-01, -6.48553392e-02],
       [ 8.63454602e-01, -5.79527124e-01,  6.47331503e-02],
       ...,
       [ 7.41611508e-01, -8.14911074e-01, -6.10944538e-05],
       [ 7.42660975e-01, -8.14911074e-01, -6.10944538e-05],
       [ 7.05090064e-01, -8.14911074e-01, -6.49636908e-02]])

In [ ]:
new_columns=a.result.columns[1:]
data_db=pd.DataFrame(a.standardscaler(),columns=new_columns)
X=data_db.copy()[['longest_strike_below_mean','longest_strike_above_mean']]
# X=data_db.copy()

In [ ]:
X.head()

In [ ]:
db=DBSCAN(eps=0.8).fit(X)
labels=db.labels_
core_samples_matrix=np.zeros_like(labels,dtype=bool)
core_samples_matrix[db.core_sample_indices_]=True


In [ ]:
colors=[plt.cm.Spectral(each) for each in np.linspace(0,1,4)]
unique_lebel=set(labels)

for i,col in zip(unique_lebel,colors):
    print(i)
    print(col)
    if i==-1:
        col=[0,0,0,1]
    class_member_class=(labels==i)
    xy=X[class_member_class & core_samples_matrix]
    plt.plot(xy.iloc[:,0],xy.iloc[:,1],'o',markeredgecolor='k',markersize=14,markerfacecolor=tuple(col))

kMeans

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
# kmeans=KMeans(n_clusters=3,random_state=0).fit(X)
# labels_kmeans=kmeans.labels_
y_pred=KMeans(n_clusters=2,random_state=0).fit_predict(X)

y_pred

In [ ]:
unique_y=set(y_pred)
for i,col in zip(unique_y,colors):
    print(i)
    xy=X[y_pred==i]
    plt.plot(xy.iloc[:,0],xy.iloc[:,1],'o',markeredgecolor='k',markerfacecolor=tuple(col) )

In [ ]:
y_pred